# Train the model

A preprocessed Nettiauto dataset is used to train the XGBoost model. The model and accompanying data is exported to separate files. The API server uses these files.

This notebook needs `nettiauto_dataset.csv` file created by `nettiauto_model.ipynb` notebook.

Running this notebook creates these files:

- `feature_names.json`
- `model.json`
- `example_X.json`
- `example_y.json`

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)


### Hide warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load preprocessed dataset

The dataset file has been created in the: `nettiauto_dataset_parsing.ipynb`

In [3]:
df = pd.read_csv('./nettiauto_dataset.csv')
# df = df.drop('color', axis=1)

df.head(25)

,make,model,color,driveType,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base
0,porsche,taycan,black,rwd,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,mercedes-benz,eqs,gray,4wd,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0
2,nissan,leaf,black,fwd,35800.0,1,10000.0,5,160.0,62.00,2.000000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,citroen,e-c4,blue,fwd,33800.0,1,17000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,hyundai,kona,white,fwd,27590.0,1,56000.0,5,150.0,64.00,2.750000,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0
5,citroen,e-c4,beige,fwd,29700.0,1,34000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,polestar,2,black,4wd,60780.0,1,7000.0,5,300.0,75.00,2.000000,0,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,0,0,0,1,0,0
7,peugeot,e-2008,orange,fwd,25790.0,1,13000.0,5,100.0,50.00,2.500000,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,bmw,ix3,white,rwd,51800.0,1,17000.0,5,210.0,80.00,2.166667,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,bmw,ix3,white,rwd,49900.0,1,25500.0,5,210.0,80.00,2.000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


## Boolean conversion

- Accessories 0/1 values (present/not-present), such as metallicColor = 1 or 0
- New columns are created for different *Make, model, color* and *driveType* strings and exclusive boolean values given for each

In [4]:
# Change Accessories 0/1 integer values to booleans
df.iloc[:, 11:] = df.iloc[:, 11:].astype(bool)

# Convert to make, model, color and dataType strings to distinct boolean values
df_encoded = pd.get_dummies(df, columns=[
    "make",
    "model",
    "color",
    "driveType"
])

df_encoded.head(25)


,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base,make_audi,make_bmw,make_byd,make_citroen,make_cupra,make_dacia,make_ds,make_fiat,make_ford,make_hyundai,make_jaguar,make_kia,make_lexus,make_maxus,make_mazda,make_mercedes-benz,make_mg,make_mini,make_muu merkki,make_nissan,make_opel,make_peugeot,make_polestar,make_porsche,make_renault,make_seat,make_skoda,make_subaru,make_tesla,make_toyota,make_volkswagen,make_volvo,model_2,model_208,model_3,model_500e,model_ariya,model_atto3,model_born,model_bz4x,model_c4,model_c40,model_citigo,model_combo-e,model_cooper,model_corsa-e,model_e-2008,model_e-208,model_e-308,model_e-berlingo,model_e-c4,model_e-c4 x,model_e-niro,model_e-soul,model_e-spacetourer,model_e-traveller,model_e-tron,model_e-tron gt,model_e-tron gt rs,model_e-up!,model_enyaq,model_eqa,model_eqb,model_eqc,model_eqe,model_eqs,model_eqs suv,model_eqv,model_euniq,model_ev6,model_ev9,model_han,model_i-pace,model_i3,model_i3s,model_i4,model_i4 m50,model_i7,model_id. buzz,model_id3,model_id4,model_id5,model_ioniq 5,model_ioniq 6,model_ioniq electric,model_ix,model_ix m60,model_ix1,model_ix3,model_kona,model_kona electric,model_leaf,model_marvel r,model_megane,model_mg4,model_mg5,model_mifa 9,model_mii electric,model_model 3,model_model s,model_model x,model_model y,model_mokka,model_mokka-e,model_mustang,model_mustang mach-e,model_muu malli,model_mx-30,model_niro,model_proace city verso ev,model_proace verso ev,model_q4 e-tron,model_q8 e-tron,model_solterra,model_spring,model_t90,model_tang,model_taycan,model_twingo,model_up!,model_ux,model_xc40,model_zafira-e,model_zoe,model_zs,color_beige,color_black,color_blue,color_brown,color_copper,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_turquoise,color_violet,color_white,color_yellow,driveType_4wd,driveType_fwd,driveType_rwd
0,101880.0,1,2100.0,5,300.0,79.00,2.416667,False,False,True,True,True,False,False,True,False,True,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,125900.0,1,30000.0,5,385.0,108.40,2.000000,False,False,True,True,True,True,False,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

## Model training

### Test split

In [5]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((3184, 182), (796, 182))

### XGBoost fit

In [6]:
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror',
                                 n_estimators=100,
                                 max_depth=8,
                                 reg_lambda=1, # näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 reg_alpha=0,# näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 num_parallel_tree=5, # random forest 🤷‍♂️
)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=5, random_state=None, ...)

### Export the model and feature list

In [25]:
import json


### Export the model
xgb_regressor.save_model("model.json")


### Export feature names
# Feature names from the model
# column_names = list(xgb_regressor.feature_names_in_)

# Feature names from the test data frame
column_names = list(X.columns)

column_names = [ { "name": x } for x in column_names ]

with open("feature_names.json", "w") as f:
    json.dump(column_names, f)

### Export an example vector and a prediction

These can be used for testing or in a mock API

In [51]:
# Take the nth single row from X and y dataframes
n = 10

# Save a single row dataframe as json
example_X = X.iloc[[n]]
example_X.to_json("example_X.json", orient="split", index=False)

# Make a prediction and save as json
predicted_y = xgb_regressor.predict(example_X)
example_y = { "price": str(predicted_y[0]) }
with open("example_y.json","w") as f:
    json.dump(example_y, f)



## Test prediction

In [7]:
y_pred = xgb_regressor.predict(X_test)


### The mean absolute error

In [8]:
mean_absolute_error(y_test, y_pred)

3583.66116471027

### The mean squared error

In [9]:
mean_squared_error(y_test, y_pred)

36862293.02566566

### R² score

In [10]:
r2_score(y_test, y_pred)

0.9516244104448957

## Test predictions by make and model

In [11]:
def predict_make_model(make, model):
    m = (X_test[f'make_{make}'] == 1) & (X_test[f'model_{model}'] == 1)
    X_id3 = X_test[m]
    y_id3 = y_test[m]

    y_id3_pred = xgb_regressor.predict(X_id3)

    print('mean_absolute_error: {}'.format(mean_absolute_error(y_id3, y_id3_pred)))
    print('r2_score: {}'.format(r2_score(y_id3, y_id3_pred)))
    print()
    for real, pred in zip(y_id3, y_id3_pred):
        print(f'{real} -> {pred} ~ {real - pred} <> {pred / real}')

### Wolkswagen ID3

In [12]:
predict_make_model('volkswagen', 'id3')

mean_absolute_error: 1875.7371794871794
r2_score: 0.8636026230790262

46600.0 -> 45640.12890625 ~ 959.87109375 <> 0.979401907859442
42340.0 -> 44583.91796875 ~ -2243.91796875 <> 1.0529975901924893
35800.0 -> 34123.546875 ~ 1676.453125 <> 0.9531717004189945
39900.0 -> 39188.9375 ~ 711.0625 <> 0.9821788847117795
34890.0 -> 32024.263671875 ~ 2865.736328125 <> 0.9178636764653195
44540.0 -> 44907.140625 ~ -367.140625 <> 1.0082429417377639
51965.0 -> 52794.38671875 ~ -829.38671875 <> 1.0159604872269796
33400.0 -> 31060.099609375 ~ 2339.900390625 <> 0.9299431020770959
41890.0 -> 38288.09765625 ~ 3601.90234375 <> 0.9140152221592266
45285.0 -> 46626.75 ~ -1341.75 <> 1.02962901623054
45900.0 -> 46501.14453125 ~ -601.14453125 <> 1.0130968307461874
46700.0 -> 52681.7890625 ~ -5981.7890625 <> 1.1280897015524625
39900.0 -> 46005.48046875 ~ -6105.48046875 <> 1.1530195606203006
42500.0 -> 46626.75 ~ -4126.75 <> 1.0971
45185.0 -> 45134.2890625 ~ 50.7109375 <> 0.9988777041606728
28400.0 -> 29312.6035156

### BMW ix1

In [13]:
predict_make_model('bmw', 'ix1')

mean_absolute_error: 2917.0558035714284
r2_score: 0.41260323100995333

69600.0 -> 71374.8359375 ~ -1774.8359375 <> 1.0255005163433908
59890.0 -> 58721.92578125 ~ 1168.07421875 <> 0.9804963396435131
70900.0 -> 67584.796875 ~ 3315.203125 <> 0.9532411406911142
65790.0 -> 61983.44921875 ~ 3806.55078125 <> 0.942140890997872
68900.0 -> 64314.765625 ~ 4585.234375 <> 0.9334508798984035
73670.0 -> 77262.453125 ~ -3592.453125 <> 1.0487641254920592
69800.0 -> 67622.9609375 ~ 2177.0390625 <> 0.9688103286174785


### Tesla 3

In [14]:
predict_make_model('tesla', 'model 3')

mean_absolute_error: 2503.8974609375
r2_score: 0.4858018739025757

45900.0 -> 48239.81640625 ~ -2339.81640625 <> 1.0509763922930284
49990.0 -> 48772.27734375 ~ 1217.72265625 <> 0.975640675010002
41880.0 -> 43931.69140625 ~ -2051.69140625 <> 1.0489897661473258
36890.0 -> 37057.0234375 ~ -167.0234375 <> 1.0045276074139333
36800.0 -> 39961.1640625 ~ -3161.1640625 <> 1.0859011973505435
45900.0 -> 45592.1015625 ~ 307.8984375 <> 0.9932919730392157
35900.0 -> 44982.08203125 ~ -9082.08203125 <> 1.2529827863857939
37890.0 -> 36726.4375 ~ 1163.5625 <> 0.9692910398522038
39999.0 -> 42820.8515625 ~ -2821.8515625 <> 1.0705480527638191
46790.0 -> 45936.1640625 ~ 853.8359375 <> 0.9817517431609318
42800.0 -> 44440.28515625 ~ -1640.28515625 <> 1.0383244195385515
39880.0 -> 39331.0859375 ~ 548.9140625 <> 0.9862358560055166
47890.0 -> 45551.65234375 ~ 2338.34765625 <> 0.951172527537064
43860.0 -> 43984.89453125 ~ -124.89453125 <> 1.0028475725319197
42990.0 -> 45755.890625 ~ -2765.890625 <> 1.064338000116

### Opel Mokka-e

In [15]:
predict_make_model('opel', 'mokka-e')

mean_absolute_error: 2116.06591796875
r2_score: 0.5139091574383912

34790.0 -> 33883.9765625 ~ 906.0234375 <> 0.9739573602328255
37780.0 -> 38610.58203125 ~ -830.58203125 <> 1.0219847017271042
40870.0 -> 40889.91796875 ~ -19.91796875 <> 1.0004873493699535
38900.0 -> 40115.1328125 ~ -1215.1328125 <> 1.0312373473650385
41660.0 -> 40562.53515625 ~ 1097.46484375 <> 0.9736566288106097
44570.0 -> 42671.3046875 ~ 1898.6953125 <> 0.957399701312542
34900.0 -> 37102.828125 ~ -2202.828125 <> 1.0631182843839542
40870.0 -> 40943.51171875 ~ -73.51171875 <> 1.0017986718558844
31990.0 -> 41161.65625 ~ -9171.65625 <> 1.2867038527664896
38470.0 -> 38610.58203125 ~ -140.58203125 <> 1.0036543288601507
32880.0 -> 33633.390625 ~ -753.390625 <> 1.022913340176399
44570.0 -> 42875.8359375 ~ 1694.1640625 <> 0.9619886905429661
29880.0 -> 34610.578125 ~ -4730.578125 <> 1.1583192143574297
40770.0 -> 36213.99609375 ~ 4556.00390625 <> 0.8882510692604857
36990.0 -> 38557.828125 ~ -1567.828125 <> 1.0423851885644768
44

### Audi e-tron

In [16]:
predict_make_model('audi', 'e-tron')

mean_absolute_error: 5450.533025568182
r2_score: 0.5681472357593222

68490.0 -> 62975.67578125 ~ 5514.32421875 <> 0.9194871628157395
62800.0 -> 61780.265625 ~ 1019.734375 <> 0.9837621914808917
47900.0 -> 54570.63671875 ~ -6670.63671875 <> 1.1392617269050105
69500.0 -> 60893.83984375 ~ 8606.16015625 <> 0.8761703574640288
51800.0 -> 50125.03125 ~ 1674.96875 <> 0.967664695945946
78790.0 -> 66779.2109375 ~ 12010.7890625 <> 0.8475594737593604
49400.0 -> 50723.8203125 ~ -1323.8203125 <> 1.0267979820344129
50900.0 -> 56750.890625 ~ -5850.890625 <> 1.1149487352652259
64999.0 -> 63588.09765625 ~ 1410.90234375 <> 0.978293476149633
51800.0 -> 62124.3671875 ~ -10324.3671875 <> 1.1993121078667954
50800.0 -> 56349.26953125 ~ -5549.26953125 <> 1.1092375891978346
